# catalog

Use **Python 3** kernel

In [18]:
import pandas as pd
pd.options.display.max_rows = 100
import sqlite3 as db
import os
from pygbif import occurrences as occ
import datetime

In [19]:
INSECTS_OF_MICRONESIA_CSV = 'observations-203843.csv'
STUDENTS = 'albi345-2021.csv'

In [20]:
def get_inat_count(data):
    inat_count = 0
    for i, x in enumerate(data['results']):
        references = x.get('references', '')
        if 'inaturalist' in references:
            inat_count += 1
        #print(i+1, references)
    return inat_count

In [21]:
if os.path.exists('iom.db'):
    os.remove('iom.db')
conn = db.connect('iom.db')

df_iom = pd.read_csv(INSECTS_OF_MICRONESIA_CSV)
df_iom.to_sql('obs', conn)

df_students = pd.read_csv(STUDENTS)
df_students.to_sql('students', conn)

In [22]:
%%time

mylist = []
for i, r in df_iom.iterrows():
    data = occ.search(limit=10000, country='GU', scientificName=r.scientific_name)
    total_count = data['count']
    if total_count > 0:
        inat_count = get_inat_count(data)
    else:
        inat_count = 0        
    if total_count == 0:
        mydict = {'user_login':r.user_login, 'id':r.id, 'total_count':0, 'inat_count':0, 'scientific_name':f"{r.scientific_name}", 
                  'GBIFchk':'WARNING: No previous GBIF records for this taxon on Guam'}
    else:
        if inat_count == total_count:
            mydict = {'user_login':r.user_login, 'id':r.id, 'total_count':total_count, 'inat_count':inat_count, 'scientific_name':f"{r.scientific_name}", 
                      'GBIFchk':f'WARNING: {total_count} previous GBIF records for this taxon on Guam; but they are all from iNaturalist'}
        else:
            mydict = {'user_login':r.user_login, 'id':r.id, 'total_count':total_count, 'inat_count':inat_count, 'scientific_name':f"{r.scientific_name}", 
                      'GBIFchk': f'OK: {total_count} previous GBIF records for this taxon on Guam'}
    print(mydict)
    mylist.append(mydict)

ConnectionError: HTTPSConnectionPool(host='api.gbif.org', port=443): Max retries exceeded with url: /v1/occurrence/search?scientificName=Phaneroptera+furcifera&country=GU&limit=10000&offset=0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb1335486d0>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [23]:
df_GBIFcheck = pd.DataFrame.from_dict(mylist)

In [24]:
df_GBIFcheck.to_sql('GBIFcheck', conn)

In [25]:
print('FINISHED')

FINISHED


In [26]:
len(df_iom.columns)

66

In [27]:
df_iom1 = df_iom.dropna(how='all', axis=1)
len(df_iom1.columns)

50

In [28]:
df_iom1.columns

Index(['id', 'observed_on_string', 'observed_on', 'time_observed_at',
       'time_zone', 'user_id', 'user_login', 'created_at', 'updated_at',
       'quality_grade', 'license', 'url', 'image_url', 'description',
       'num_identification_agreements', 'num_identification_disagreements',
       'captive_cultivated', 'oauth_application_id', 'place_guess', 'latitude',
       'longitude', 'positional_accuracy', 'public_positional_accuracy',
       'taxon_geoprivacy', 'coordinates_obscured', 'positioning_method',
       'positioning_device', 'place_state_name', 'place_admin1_name',
       'species_guess', 'scientific_name', 'common_name', 'iconic_taxon_name',
       'taxon_id', 'taxon_kingdom_name', 'taxon_phylum_name',
       'taxon_subphylum_name', 'taxon_class_name', 'taxon_subclass_name',
       'taxon_superorder_name', 'taxon_order_name', 'taxon_suborder_name',
       'taxon_superfamily_name', 'taxon_family_name', 'taxon_subfamily_name',
       'taxon_tribe_name', 'taxon_subtribe_name

In [29]:
df_merged = df_iom1.merge(df_GBIFcheck, on='id')

KeyError: 'id'

In [ ]:
df_merged.columns

In [ ]:
def generate_html_report(mytable_html, student_name):
    #timestamp = datetime.utcnow()
    html = f'''
        <html>
            <header>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>catalog</title>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.9.3/css/bulma.min.css">
            </header>
            <body>
                <section class="section">
                    <div class="container">
                        {mytable_html}
                    </div>
                </section>
            </body>
        </html>        
        '''
    with open(f'{student_name}.html', 'w') as f:
        f.write(html)    

In [ ]:
def create_html_table(user_login):

    df_temp = df_merged[[
        'user_login_x',
        'url',
        'taxon_order_name',
        'taxon_suborder_name', 
#         'taxon_superfamily_name', 
        'taxon_family_name',
#         'taxon_subfamily_name', 
#         'taxon_tribe_name', 
#         'taxon_subtribe_name',
        'taxon_genus_name', 
        'taxon_species_name', 
#         'taxon_subspecies_name',        
        'scientific_name_x',
        'GBIFchk']][df_merged.user_login_x==user_login]

    df_temp.sort_values([
        'taxon_order_name',
        'taxon_suborder_name', 
#         'taxon_superfamily_name', 
        'taxon_family_name',
#         'taxon_subfamily_name', 
#         'taxon_tribe_name', 
#         'taxon_subtribe_name',
        'taxon_genus_name', 
        'taxon_species_name'], inplace=True)
    
    df_temp.columns = ['user', 'url', 'order', 'suborder', 'family', 'genus', 'species','scientific_name','GBIFchk']

    return df_temp.to_html(index=False)
    
# create_html_table('entomologist_of_mystery')

In [ ]:
for i,r in df_students.iterrows():
    print(r['name'])
    mytable_html = create_html_table(r['inat_user_login'])
    generate_html_report(mytable_html, r['name'])

In [ ]:
for i,r in df_students.iterrows():
    name = r['name']
    s = f'<a href="https://aubreymoore.github.io/inat_labels/{name.replace(" ","%20")}.html">{name}</a><br>'
    print(s)